In [ ]:
# Part 1: Sparse Keypoint Detection and Tracking (SIFT)

# Importing necessary libraries for tasks
import zipfile # For extracting 'data.zip'
import os # For image file handling
import cv2 # for SIFT and keypoint matching
import numpy as np # For calculations
import matplotlib.pyplot as plt  # For visualisations

# In order for extraction below to work, need to first upload 'data.zip' to Files
# Extracting 'data.zip' to a 'data' folder with all the frames in it
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
# All three tasks (keypoint extraction, matching, and visualisation) completed in this cell

# Initialising SIFT detector
sift = cv2.SIFT_create()

"""
Initialising BFMatcher with L2 norm (for SIFT).
cv2.NORM_L2: Euclidean distance; crossCheck=True: mutual best match
"""
bf = cv2.BFMatcher()

# Output folder to store every 5th frame
sift_results = 'SIFT_Results/'
os.makedirs(sift_results, exist_ok=True) # creating folder if it does not yet exist

for i in range(0,69): # 0 to 68
  f1_path = os.path.join(f'data/frame_{i}.jpg')
  f2_path = os.path.join(f'data/frame_{i+1}.jpg')

  # Loading the frames
  frame1_BGR = cv2.imread(f1_path)
  frame2_BGR = cv2.imread(f2_path)

  # Converting the frame to grayscale for SIFT
  frame1_gray = cv2.cvtColor(frame1_BGR, cv2.COLOR_BGR2GRAY)
  frame2_gray = cv2.cvtColor(frame2_BGR, cv2.COLOR_BGR2GRAY)

  # If either frame is not loaded, raise an error
  if frame1_gray is None or frame2_gray is None:
    raise ValueError("Error loading frame. Check that the file path exists.")

  # Task 1: Extracting the keypoints and descriptors for both frames
  f1_kp, f1_des = sift.detectAndCompute(frame1_gray, None)
  f2_kp, f2_des = sift.detectAndCompute(frame2_gray, None)

  # Task 2: Keypoint Matching
  knn_matches = bf.knnMatch(f1_des, f2_des, k=2)
  #print("Total match count: ", len(knn_matches)) # Using for Part 1 report analysis

  # Applying Lowe's ratio test to filter good matches
  # Referred to: https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html
  good_matches = []
  for m1, m2 in knn_matches:
    if m1.distance < (m2.distance * 0.75):
      good_matches.append(m1)
  #print("Total good match count: ", len(good_matches)) # Using for Part 1 report analysis

  # Sorting and truncating to best 800 matches for visualisation (like in Week 6 tutorial)
  top_matches = sorted(good_matches, key=lambda x: x.distance)
  top_800_matches = top_matches[:800]

  #arrow_count = 0 # Using to check how many arrows of the 800 arrows are omitted from each frame's visualisation

  # Task 3: Visualisation
  for tm in top_800_matches:
    # Extracting keypoints from frame1 and frame2
    f1_pt = tuple(map(int, f1_kp[tm.queryIdx].pt))
    f2_pt = tuple(map(int, f2_kp[tm.trainIdx].pt))
    # Calculating the distance between the keypoints
    dist_x = f2_pt[0] - f1_pt[0]
    dist_y = f2_pt[1] - f1_pt[1]
    # Calculating the magnitude
    magnitude = np.sqrt(dist_x**2 + dist_y**2)
    # Only overlaying the small arrows on the current frame (frame2)
    if magnitude < 50: # 775-800 arrows are visualised depending on the frame; just filters out the rare extremely large arrows
      cv2.arrowedLine(frame2_BGR, f1_pt, f2_pt, (0, 255, 0), 2, tipLength=0.4)
      #arrow_count += 1

  # Visualising the current frame with the arrows
  plt.figure(figsize=(12, 6))
  plt.imshow(cv2.cvtColor(frame2_BGR, cv2.COLOR_BGR2RGB))
  #print(arrow_count)
  plt.axis("off")
  plt.title(f"SIFT Feature Matches: frame_{i}.jpg -> frame_{i+1}.jpg")
  plt.show()

  # Saving every fifth frame's resulting image
  if (i+1) % 5 == 0:
    filename = os.path.join(sift_results, f'frame_{i+1}.jpg')
    cv2.imwrite(filename, frame2_BGR)

## The cell below is only needed if running the code in Google Colab rather than locally on Jupyter Notebook.

In [ ]:
# Downloading the saved 'SIFT_Results' from Google Colab to my local machine

from google.colab import files
import shutil

# Converting the file into a zip file
shutil.make_archive('SIFT_Results', 'zip', 'SIFT_Results')

# Downloading the zip file to my local machine
files.download('SIFT_Results.zip')